### Prediksi Harga Rumah di Kabupaten Tangerang
#### Daffa Kaisha Pratama Chandra - 55770
---
##### Import library yang dibutuhkan

In [1]:
# Library dasar
import pandas as pd
import numpy as np

# Library untuk visualisasi
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Data Pre-processing
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold

# Machine Learning
from sklearn.linear_model import LinearRegression # Regresi linear
from sklearn.ensemble import RandomForestRegressor # Random forest regressor
from xgboost import XGBRegressor # XGBoost

# Evaluation
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

---
#### Baca data harga rumah di tangerang
Deskripsi singkat mengenai dataset: <br>
Dataset ini diambil dengan cara <i> web scraping </i> pada situs realoka.com. <br>
Dataset ini masih berupa data mentah sehingga harus di proses terlebih dahulu.<br>
Dataset ini berisi data harga rumah dan spesifikasi rumah di wilayah Kabupaten Tangerang.

In [3]:
# Baca data
rumah = pd.read_csv("rumah_kab_tangerang.csv", engine='pyarrow')
rumah.head()

,nama_rumah,alamat,harga,deskripsi,Tayang Sejak,Luas Tanah,Luas Bangunan,Kamar Tidur,Kamar Mandi,Sertifikat,...,Daya Listrik1,Parkir1,Kolam Renang1,K. Pembantu1,Air PAM1,Balkon1,One Gate1,Pengamanan1,Wilayah,url
0,"New Cluster Puri Jaya, Cluster Maleo, dekat Ko...","Puri Jaya Pasar Kemis Nur Sodik, Tangerang Kab",440.000.000,Deskripsi dan SpesifikasiPuri Jaya adalah PERU...,5 hari yang lalu,60 m2,31 m2,2,1,SHM - Sertifikat Hak Milik,...,2200 VA,1 mobil,1.0,1.0,1.0,1.0,1.0,1.0,Kabupaten Tangerang,https://www.realoka.com/408932-new-cluster-pur...
1,Cluster Tanamera Residence dekat Bandara Soett...,"Jalan Gatot Subroto Tanah Merah Sepatan, Tang...",297.000.000,Deskripsi dan SpesifikasiCLUSTER TANAMERA RESI...,6 hari yang lalu,60 m2,30 m2,2,1,SHM - Sertifikat Hak Milik,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,Kabupaten Tangerang,https://www.realoka.com/410318-cluster-tanamer...
2,Rumah Murah dekat Puspem Bayar 2juta Akad Ruma...,"Jalan Raya Puspem Tigaraksa Matagara, Tangera...",320.400.000,Deskripsi dan SpesifikasiCLUSTER RUMAHKU MATAG...,6 hari yang lalu,60 m2,30 m2,2,1,SHM - Sertifikat Hak Milik,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,Kabupaten Tangerang,https://www.realoka.com/410317-rumah-murah-dek...
3,"Cluster Terbaru Puri, Cluster Maleo, Banyak Pr...","Puri Jaya Pasar Kemis, Tangerang Kab",500.000.000,Deskripsi dan SpesifikasiPuri Jaya Pasar Kemis...,2 minggu yang lalu,60 m2,46 m2,2,2,SHM - Sertifikat Hak Milik,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,Kabupaten Tangerang,https://www.realoka.com/412981-cluster-terbaru...
4,Rumah di Kelapa Dua Serpong 2 Lantai SIAP HUNI,"Kelapa Dua Tangerang, Tangerang Kab",950.000.000,Deskripsi dan Spesifikasi*Rumah mewah 2 lantai...,46 menit yang lalu,88 m2,79 m2,2,2,SHM - Sertifikat Hak Milik,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,Kabupaten Tangerang,https://www.realoka.com/287076-rumah-di-kelapa...


In [4]:
rumah.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4644 entries, 0 to 4643
Data columns (total 47 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   nama_rumah      4644 non-null   object 
 1   alamat          4644 non-null   object 
 2   harga           4644 non-null   object 
 3   deskripsi       4644 non-null   object 
 4   Tayang Sejak    4644 non-null   object 
 5   Luas Tanah      4644 non-null   object 
 6   Luas Bangunan   4644 non-null   object 
 7   Kamar Tidur     4644 non-null   int64  
 8   Kamar Mandi     4644 non-null   int64  
 9   Sertifikat      4644 non-null   object 
 10  Kondisi         4644 non-null   object 
 11  Kelengkapan     4644 non-null   object 
 12  Jumlah Lantai   4644 non-null   int64  
 13  Menghadap       4644 non-null   object 
 14  Daya Listrik    4386 non-null   object 
 15  Parkir          4644 non-null   object 
 16  Kolam Renang    516 non-null    float64
 17  Air PAM         1548 non-null   f